# 일반적인 그리드

In [6]:
def set_cover(U, s):
    uncovereded = U
    covered = []

    while uncovereded:
        greedy_set = max(s, key=lambda x: len(uncovereded.intersection(x)))
        covered.append(greedy_set)
        uncovereded -= greedy_set
    return covered

In [7]:
U = {1, 2, 3, 4, 5, 6, 7, 8, 9}
S = [{1, 2, 3, 4, 5, 6}, {1, 2, 4, 5, 7}, {3, 5, 6, 8, 9}, {6, 9}]
result = set_cover(U, S)
print(result)

[{1, 2, 3, 4, 5, 6}, {3, 5, 6, 8, 9}, {1, 2, 4, 5, 7}]


## Improved_set_cover(수정 필요)

In [28]:
def set_cover_improved(U, S):
    # 선택된 부분 집합들을 담을 리스트
    selected_subsets = []
    
    # 모든 집합이 커버될 때까지 반복
    while len(U) > 0:
        # 최대 커버 갯수를 저장할 변수
        max_cover_count = 0
        
        # 최대 커버 갯수를 가지는 부분 집합을 선택할 변수
        selected_subset = None
        
        # 모든 부분 집합에 대해 반복
        for subset in S:
            # 현재 부분 집합이 선택되었는지 확인
            if subset in selected_subsets:
                continue

            # 부분 집합이 커버하는 요소의 개수
            cover_count = len(set(subset).intersection(U))
            
            # 최대 커버 갯수보다 큰 경우 업데이트
            if cover_count > max_cover_count:
                max_cover_count = cover_count
                selected_subset = subset
            elif cover_count == max_cover_count and selected_subset is not None:
                # 처음 시행횟수에는 앞의 인덱스를 선택(그리디와 동일한 방식으로 동작하기 위해 기입하지 않아도 되긴함)
                if len(selected_subsets) == 0:
                    idx = S.index(selected_subset)
                    if S.index(subset) < idx:
                        selected_subset = subset
                else:
                    # 2번째 이후부터는 최초의 최대 커버 요소를 선택
                    first_subset = selected_subsets[0]
                    first_cover_count = len(set(first_subset).intersection(U))
                    if cover_count > first_cover_count:
                        selected_subset = subset
        
        # 선택된 부분 집합을 추가
        selected_subsets.append(selected_subset)
        
        # 선택된 부분 집합으로부터 U 업데이트
        U = U.difference(selected_subset)

    # 모든 집합을 커버완료후 중복되지 않은 고유값 찾기
    unique_values = set()
    for subset in selected_subsets:
        unique_values.update(set(subset))

    # 선택된 집합들 중에 고유값을 포함하지 않는 집합 제거
    for subset in selected_subsets:
        if not unique_values.isdisjoint(set(subset)):
            selected_subsets.remove(subset)
            break

    return selected_subsets

In [29]:
U = {1, 2, 3, 4, 5, 6, 7, 8, 9}
S = [{1, 2, 3, 4, 5, 6}, {1, 2, 4, 5, 7}, {3, 5, 6, 8, 9}, {6, 9}]
opt = set_cover_improved(U, S)
print(opt)

[{3, 5, 6, 8, 9}, {1, 2, 4, 5, 7}]


## 최적해에 더 근접하는 개선된 셋커버

In [30]:
U = {1, 2, 3, 4, 5, 6, 7, 8, 9,10}
S = [{1, 3, 5, 7, 9}, {2, 4, 6, 8, 10}, {1, 2, 3, 4, 5 ,6}]
print('개선된 셋커버: ', set_cover_improved(U, S))
print('일반적인 셋커버: ', set_cover(U, S))

개선된 셋커버:  [{2, 4, 6, 8, 10}, {1, 3, 5, 7, 9}]
일반적인 셋커버:  [{1, 2, 3, 4, 5, 6}, {1, 3, 5, 7, 9}, {2, 4, 6, 8, 10}]


In [34]:
def calculateSubsetFrequencies(sets):
    frequencies = {}
    for subset in sets:
        for element in subset:
            if element in frequencies:
                frequencies[element] += 1
            else:
                frequencies[element] = 1

    return frequencies


def calculateSubsetInitialSizes(sets):
    initial_sizes = {}
    for i, subset in enumerate(sets):
        initial_sizes[i+1] = len(subset)

    return initial_sizes


def findOptimalCover(sets):
    universe = set()
    for subset in sets:
        universe |= set(subset)

    optimalCover = []
    while universe:
        bestSubset = None
        bestCovered = set()
        for subset in sets:
            covered = set(subset) & universe
            if len(covered) > len(bestCovered):
                bestSubset = subset
                bestCovered = covered

        if bestSubset is None:
            break

        if len([subset for subset in sets if len(set(subset) & universe) == len(bestCovered)]) > 1:
            initialSizes = calculateSubsetInitialSizes(sets)
            bestSubset = max([subset for subset in sets if len(set(subset) & universe) == len(bestCovered)],
                             key=lambda x: initialSizes[sets.index(x)+1])
        
        optimalCover.append(bestSubset)
        universe -= bestCovered
        sets.remove(bestSubset)

    frequencies = calculateSubsetFrequencies(optimalCover)
    updatedOptimalCover = []
    deletedOptimalCover = []

    for subset in optimalCover:
        if any(frequencies[element] == 1 for element in subset):
            updatedOptimalCover.append(subset)
        elif all(frequencies[element] != 1 for element in subset):
            deletedOptimalCover.append(subset)

    return updatedOptimalCover, deletedOptimalCover

sets = [
        [1, 3, 5, 7, 9],
        [2, 4, 6, 8, 10],
        [1, 2, 3, 4, 5 ,6]
    
    ]

optimalCover = findOptimalCover(sets)
print(len(optimalCover))



2


## 그리디와 improved 그리디 난수환경 비교

In [9]:
import random
from collections import defaultdict

def set_cover(universe, sets):
    uncovered = set(universe)
    cover = []
    while uncovered:
        best_set = max(sets, key=lambda x: len(uncovered.intersection(x)))
        cover.append(best_set)
        uncovered -= best_set
    return cover

def calculateSubsetFrequencies(sets):
    frequencies = {}
    for subset in sets:
        for element in subset:
            if element in frequencies:
                frequencies[element] += 1
            else:
                frequencies[element] = 1

    return frequencies


def calculateSubsetInitialSizes(sets):
    initial_sizes = {}
    for i, subset in enumerate(sets):
        initial_sizes[i+1] = len(subset)

    return initial_sizes


def set_cover_improved(universe, sets):
    universe = universe
    for subset in sets:
        universe |= set(subset)

    optimalCover = []
    while universe:
        bestSubset = None
        bestCovered = set()
        for subset in sets:
            covered = set(subset) & universe
            if len(covered) > len(bestCovered):
                bestSubset = subset
                bestCovered = covered

        if bestSubset is None:
            break

        if len([subset for subset in sets if len(set(subset) & universe) == len(bestCovered)]) > 1:
            initialSizes = calculateSubsetInitialSizes(sets)
            bestSubset = max([subset for subset in sets if len(set(subset) & universe) == len(bestCovered)],
                             key=lambda x: initialSizes[sets.index(x)+1])
        
        optimalCover.append(bestSubset)
        universe -= bestCovered
        sets.remove(bestSubset)

    frequencies = calculateSubsetFrequencies(optimalCover)
    updatedOptimalCover = []
    deletedOptimalCover = []

    for subset in optimalCover:
        if any(frequencies[element] == 1 for element in subset):
            updatedOptimalCover.append(subset)
        elif all(frequencies[element] != 1 for element in subset):
            deletedOptimalCover.append(subset)

    return updatedOptimalCover


def generate_sets(num_points, probability, num_sets):
    universe = set(range(1, num_points + 1))
    sets = []
    for _ in range(num_sets):
        new_set = set()
        for point in range(1, num_points + 1):
            if random.random() <= probability:
                new_set.add(point)
        sets.append(new_set)
    return sets, universe

num_points = 100  # 요소 개수 
num_sets = 100  # 부분 집합 개수
probability = 0.3  # 부분 집합이 집합의 각 요소를 포함할 확률
#### 확률이 30퍼이기 때문에 40개를 포함할 수도 있음
n = 1000  # 반복 실행 횟수

greedy_set_totals = defaultdict(int)
improved_set_totals = defaultdict(int)

for _ in range(n):
    sets, universe = generate_sets(num_points, probability, num_sets)
    greedy_cover = set_cover(universe, sets)
    improved_cover = set_cover_improved(universe, sets)
    greedy_set_totals[len(greedy_cover)] += 1
    improved_set_totals[len(improved_cover)] += 1

sorted_greedy_totals = sorted(greedy_set_totals.items(), key=lambda x: x[0])
sorted_improved_totals = sorted(improved_set_totals.items(), key=lambda x: x[0])

for length, count in sorted_greedy_totals:
    print(f'set_total이 {length}인 경우의 개수 (그리디 알고리즘): {count}')

print("="*50)
    
for length, count in sorted_improved_totals:
    print(f'set_total이 {length}인 경우의 개수 (개선된 알고리즘): {count}')

set_total이 5인 경우의 개수 (그리디 알고리즘): 51
set_total이 6인 경우의 개수 (그리디 알고리즘): 876
set_total이 7인 경우의 개수 (그리디 알고리즘): 73
set_total이 4인 경우의 개수 (개선된 알고리즘): 3
set_total이 5인 경우의 개수 (개선된 알고리즘): 151
set_total이 6인 경우의 개수 (개선된 알고리즘): 808
set_total이 7인 경우의 개수 (개선된 알고리즘): 38
